In [1]:
import setup
setup.init_django(project_name='mirhome')

/home/mirsquared/Documents/WorkEnv/django_web_scraping/src/nbs


In [9]:
import helpers

In [10]:
from bs4 import BeautifulSoup

In [11]:
url = "https://www.amazon.com/PlayStation%C2%AE5-Digital-slim-PlayStation-5/dp/B0CL5KNB9M/ref=sr_1_3?crid=B7GWFZXZ2OED&dib=eyJ2IjoiMSJ9.kgp6If9Ie8zGHwBo-0htBLyQbbKjs5VuqpcJV5opH4mRqQT9y1GDUhgEGC4Ze5c7iOpklu5U3l_vF3hGTmGfZf8jvVY7cSGvtmhRbSth2-wUchP4cPB4bCopxZnBPpqLbX4wU-JZkepl_i4fGdXQJXUMLc256FqdCdlbjr6ZMyFHhWIJq2G38fcfQx3z9RS1e48jNXaYXv1rWtJ3Y30-OZP-ckGz15zF5vR6k6z6G6c.HDrf64xu0Nz7DYLZvUdGglWizRZpAXBxsxbsgsW26Tc&dib_tag=se&keywords=ps5&qid=1709675943&sprefix=ps5%2Caps%2C190&sr=8-3"

/home/mirsquared/Documents/WorkEnv/django_web_scraping/src


In [12]:
url

In [13]:


html = helpers.scrape(url=url, solve_captcha=False)
data = helpers.extract_amazon_product_data(html)



Connecting to Scraping Browser...
Connected! Navigating to https://www.amazon.com/PlayStation%C2%AE5-console-slim-PlayStation-5/dp/B0CL61F39H/ref=sr_1_1?dib=eyJ2IjoiMSJ9.F-c6KBJm1tKH0t43iBScfCfEIEFn9fme4tekt5SU7EjryaAMWaw68ZfG8m3hfxXlOne3XBl9wMtDuxpR1Dh43aBybWzasNjXqd2ynwBmGft0i9lDFlGCZaPpz4QbEtxb6EQVc3yPb6F8KVifO6vGSkwV-VVnmqtRohIchqgVB1tr6luX7FCinpCGInQg1yU1vSuKnfH6nOP0ie4q8KhXVvLdPfGo3RRrPMf_0qp_wJk.v3FhmRXudDCFUMB2J4tD4vYitrMWo7gMje6bx0NjomU&dib_tag=se&keywords=playstation+5&qid=1727486113&sr=8-1
Navigated! Scraping page content...


```html
{'title': 'PlayStation®5 Digital Edition (slim)',
 'price_raw': '449.',
 'price_text': '449.',
 'price': 449.0,
 'metadata': [{'ASIN': 'B0CL5KNB9M'},
  {'Release date': 'November 24, 2023'},
  {'Customer Reviews': '4.7  4.7 out of 5 stars    \n    3,538 ratings   \n\n\n 4.7 out of 5 stars'},
  {'Best Sellers Rank': '#216 in Video Games (See Top 100 in Video Games)  #2 in PlayStation 5 Consoles'},
  {'Product Dimensions': '17 x 15 x 6 inches; 8.9 Pounds'},
  {'Type of item': 'Video Game'},
  {'Item model number': 'CFI-2000'},
  {'Item Weight': '8.9 pounds'},
  {'Manufacturer': 'Sony'},
  {'Country of Origin': 'China'},
  {'Batteries': '1 Lithium Ion batteries required. (included)'},
  {'Date First Available': 'November 24, 2023'}],
 'description': ' \n Play Like Never Before. The PS5 Digital Edition unleashes new gaming possibilities that you never anticipated. Experience lightning fast loading with an ultra-high speed SSD, deeper immersion with support for haptic feedback, adaptive triggers, and 3D Audio*,and an all-new generation of incredible PlayStation® games. PS5 Digital Edition is an all-digital version of the PS5 console with no disc drive. Sign into your account for PlayStation Network and go to PlayStation Store to buy and download games (Account for PlayStation Network required). Lightning Speed - Harness the power of a custom CPU, GPU, and SSD with Integrated I/O that rewrite the rules of what a PlayStation console can do. Stunning Games - Marvel at incredible graphics and experience new PS5 features. Play a back catalog of supported PS4 games. Breathtaking Immersion - Discover a deeper gaming experience with support for haptic feedback, adaptive triggers, and 3D Audio technology. *3D audio via built-in TV speakers or analog/USB stereo headphones. Set up and latest system software update required.  ',
 'feature_bullets': ' \n About this item    Model Number CFI-2000    Includes DualSense Wireless Controller, 1TB SSD, 2 Horizontal Stand Feet, HDMI Cable, AC power cord, USB cable, printed materials, ASTRO’s PLAYROOM (Pre-installed game)    Vertical Stand sold seperately    \n',
 'rating': {'average': 4.7, 'count': 3538}}
```

In [15]:
def find_product_table_data(html):
    soup = BeautifulSoup(html)
    product_data = soup.find('div', id='prodDetails')
    if product_data is None:
        return []
    table = product_data.find('table')
    columns = [f"{x.text}".strip() for x in table.find_all('th')]
    table_data=[]
    for i, row in enumerate(table.find_all('tr')):  # [1:] to skip the header row
        # Get all cells in the row
        cells = row.find_all('td')
        # Create a dictionary for the current row, mapping header to cell data
        row_data = {columns[i]: f'{cell.text}'.strip() for cell in cells}
        # Add the dictionary to your list
        table_data.append(row_data)
    return table_data

def find_product_rating(html):
    soup = BeautifulSoup(html)
    average_rating = soup.find(id='averageCustomerReviews').find_all("span", class_='a-size-base')[0].text.strip()
    average_rating = "".join([x for x in f"{average_rating}".strip() if x.isdigit() or x == '.'])
    average_rating = float(average_rating)
    rating_data = soup.find(id='acrCustomerReviewText').text
    rating_count = int(''.join([x for x in rating_data if x.isdigit()]))
    rating_count
    return {
        'average': average_rating,
        'count': rating_count,
    }

def extract_amazon_product_data(html):
    soup = BeautifulSoup(html)
    productTitle = soup.find('span', id='productTitle')
    productTitleText = f"{productTitle.text}".strip()
    productPrice = soup.find_all('span', class_='a-price-whole')[0]
    productPrice = f"{productPrice.text}".strip()
    productPriceText = "".join([x for x in productPrice if x.isdigit() or x == '.'])
    productPriceNum = float(productPriceText)
    try:
        productDescription = soup.find('div', id='productDescription').text
    except:
        productDescription = ''
    featureBullets = soup.find('div', id='feature-bullets').text
    return {
        'title': productTitleText,
        'price_raw': productPrice,
        'price_text': productPriceText,
        'price': productPriceNum,
        'metadata': find_product_table_data(html),
        'description': productDescription,
        'feature_bullets': featureBullets,
        'rating': find_product_rating(html)
    }



In [19]:
html = helpers.scrape(url=url, solve_captcha=False)

'PlayStation®5 console (slim)'

In [2]:
# with open('output.html', 'w+') as f:
#     f.write(html)

In [23]:
data = extract_amazon_product_data(html)
data

'$49.99'